In [ ]:
%pip install --upgrade openai

In [14]:
import openai
import psycopg2
import pandas as pd
from sentence_transformers import SentenceTransformer
# filepath ='/home/azureuser/cloudfiles/code/Users/supakit/SimilaritySearch/Mon/'

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
openai.api_type = "azure"
openai.api_base = "https://fusion-gpt-training.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "a83037e20d3d4d7982b0c52080e730fb"

In [7]:
host = "pureelawvec.postgres.database.azure.com"
dbname = "postgres"
user = "pgadmin"
password = "Puree1234"
sslmode = "require"
# Construct connection string

In [9]:
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
conn = psycopg2.connect(conn_string) 
print("Connection established")
cursor = conn.cursor()
# Fetch all rows from table

Connection established


In [ ]:
import pandas as pd
import os

df = pd.read_csv("criminal-datasets.csv")
df['text'] = df['text'].astype(str) + df['notes'].astype(str)
df['text'] = df['text'].str.replace('nan', '')
df = df.drop(['article','notes'], axis = 1)
df = df.drop(df.index[0:9])
df = df.reset_index()
df = df.drop('index', axis = 1)
df

In [10]:
import requests, uuid, json
# You can pass more than one object in body.
def thai2eng(input):
    key = "041e6b79b96146eabafaf524d78911be"
    endpoint = "https://api.cognitive.microsofttranslator.com"
    location = "southeastasia"
    path = '/translate'
    constructed_url = endpoint + path

    params = {
        'api-version': '3.0',
        'from': 'th',
        'to': ['en']
    }

    headers = {
        'Ocp-Apim-Subscription-Key': key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }
    # translated = []
    # for string in input:
    body = [{
        'text': input
    }]
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    # translated.append(response[0]["translations"][0]['text'])
    # print(json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))
    # print(response[0]["translations"][0]['text'])
    return response[0]["translations"][0]['text']

def eng2thai(input):
    key = "041e6b79b96146eabafaf524d78911be"
    endpoint = "https://api.cognitive.microsofttranslator.com"
    location = "southeastasia"
    path = '/translate'
    constructed_url = endpoint + path

    params = {
        'api-version': '3.0',
        'from': 'en',
        'to': ['th']
    }

    headers = {
        'Ocp-Apim-Subscription-Key': key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }
    # translated = []
    # for string in input:
    body = [{
        'text': input
    }]
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    return response[0]["translations"][0]['text']

In [44]:
import pandas as pd
import re

df = pd.read_csv("law.csv")
df['Content'] = df['Content'].str.replace("\n", " ")
# Extract sections using regular expressions
pattern = r'Section \d+\..*?(?=Section \d+\.|$)'
df['Section'] = df['Content'].apply(lambda x: re.findall(pattern, x, re.DOTALL))
# Explode the list of sections into separate rows
df = df.explode('Section')
df_text = df.drop('Content', axis= 1)
# # Reset index
df_text.reset_index(drop=True, inplace=True)
df = df_text.dropna()
# for row in df_text.Content:
#     if row < 20:
#         df_text.drop(row)

In [ ]:
import numpy as np
x = np.squeeze(df.Section.values)
sentence_list = [x for x in x]
sentence_list

In [ ]:
translated = []
for item in sentence_list:
    translated.append(eng2thai(item))
translated

In [ ]:
from sentence_transformers import SentenceTransformer
embed_list = []
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentence_list)
embed_list.append(embeddings.tolist())
print(embeddings)


In [ ]:
try:
    # cursor.execute("CREATE EXTENSION IF NOT EXISTS pg_trgm;")
    cursor.execute("DROP TABLE Thai_Law")
    cursor.execute("DROP TABLE Eng_Law")
    cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    cursor.execute(""" 
    CREATE TABLE Criminal_Law(
        id INTEGER PRIMARY KEY,
        embedding vector,
        text_eng TEXT,
        text_thai TEXT
    );""")
    for i in range(len(sentence_list)):  
        cursor.execute("INSERT INTO Criminal_Law(id, embedding, text_eng, text_thai) VALUES (%s, %s, %s, %s);",   
                    (i, embed_list[0][i], sentence_list[i], translated[i]))
        print(sentence_list[i]) 
        print(embed_list[0][i])
        conn.commit()  
except Exception as e:
        print(f"Error: {e}")
        conn.rollback()

In [12]:
# def text2vec(input):
#     # input = translator(input)
#     embedding = openai.Embedding.create(input = input, engine = "text-embedding-ada-002")
#     return embedding['data'][0]['embedding']

def text2vec(input):
    embed_list = []
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    embeddings = model.encode(input)
    embed_list.append(embeddings.tolist())
    return embed_list[0]

def search_vectordb(question):
    search_text = []
    try:
         cursor.execute(f"""SELECT text_thai, cosine_distance('{text2vec(question)}', embedding) as cosign
         FROM Criminal_Law
         ORDER BY cosine_distance('{text2vec(question)}', embedding) ASC LIMIT 7
         """)
         query = cursor.fetchall()
         text_eng_results = [row[0] for row in query] 
         return text_eng_results
    except Exception as e:  
            print(f"Error: {e}")
            conn.rollback()
    return "null"

#WHERE cosine_distance('{text2vec(question)}', embedding) < {start}

In [11]:
Lawyer_prompt = "LawBot, a GPT-based AI assistant designed specifically for answering legal questions and explaining complex legal terms.\
        Analysis the Question sector and provide a comprehensive and briefly answer in Answer section.The answer must address all aspects\
        of the Question and support your argument with appropriate legal references in Context section. Only answered by thai language only "

In [118]:
setting = [{"role":"system","content":""},{"role":"user","content":""}]
def gpt3_completion(user_input, engine='gpt-35-turbo-16k', temp=0.5, top_p=0.95, tokens=2000, freq_pen=0.0, pres_pen=0.2):
    setting.append({'role':'system',"content":  
      Lawyer_prompt + ''.join(search_vectordb(thai2eng(gpt3_sum(user_input))))})
    # setting.append({'role':'user',"content":f"{gpt3_sum(user_input)}"})
    response = openai.ChatCompletion.create(engine=engine, temperature=temp, max_tokens=tokens,
      top_p=top_p, frequency_penalty=freq_pen, presence_penalty=pres_pen, messages = setting)
    print("LawBot:", response['choices'][0]['message']['content'])
    
def gpt3_sum(user_input, engine='gpt-35-turbo', temp=0.5, top_p=0.95, tokens=2000, freq_pen=0.0, pres_pen=0.2):
    setting.append({'role':'system',"content":"Summary the user input as a bullet , use only thai language"})
    setting.append({'role':'user',"content":f"{user_input}"})
    response = openai.ChatCompletion.create(engine=engine, temperature=temp, max_tokens=tokens,
      top_p=top_p, frequency_penalty=freq_pen, presence_penalty=pres_pen, messages = setting)
    # print("SumBot:", response['choices'][0]['message']['content'])
    return response['choices'][0]['message']['content']

# while True:
#   # เผาบ้านเรือนมีความผิดตามข้อไหน
#   # ป้องกันตัวเองอย่างไรไม่ให้ผิดกฏหมาย
#   user_input = input()
#   print("User:", user_input)
#   if user_input == "quit":
#     break
#   response = gpt3_completion(thai2eng(user_input))

In [122]:
x = gpt3_sum("ป้องกันตัวเองอย่างไรไม่ให้ผิดกฏหมาย")

In [136]:
def update_setting(role, content):  
    setting.append({"role": role, "content": content})  
  
def get_response_from_gpt3(user_input, engine='gpt-35-turbo-16k', temp=0.5, top_p=0.95, tokens=2000, freq_pen=0.0, pres_pen=0.2):  
    update_setting('user', user_input)  
    lawyer_prompt_content = Lawyer_prompt + ''.join(search_vectordb(thai2eng(gpt3_sum(user_input))))  
    update_setting('system', lawyer_prompt_content)  
  
    response = openai.ChatCompletion.create(engine=engine, temperature=temp, max_tokens=tokens,  
      top_p=top_p, frequency_penalty=freq_pen, presence_penalty=pres_pen, messages = setting)  
    return response['choices'][0]['message']['content']  
  
# usage  
user_input = "อยากฆ่าคนทำได้ไหม"  
print("LawBot:", get_response_from_gpt3(user_input))  


LawBot: ในกรณีที่คุณมีความคิดอยากฆ่าคน คุณต้องระวางโทษตามกฎหมายไทย ภายใต้มาตรา 290 ของประมวลกฎหมายอาญา ผู้ที่ทำให้ผู้อื่นถึงแก่ความตายโดยทำร้ายร่างกายของบุคคลนั้นโดยไม่มีเจตนาทำให้ผู้อื่นถึงแก่ความตาย จะต้องถูกลงโทษจำคุกตั้งแต่ 3 ปีถึง 15 ปี อย่างไรก็ตาม หากการกระทำของคุณเป็นผลให้เกิดความเสียหายต่อผู้อื่นโดยร้ายแรง คุณอาจถูกลงโทษจำคุกตั้งแต่ 1 ปีถึง 20 ปี และต้องรับโทษปรับตั้งแต่ 2,000 บาทถึง 20,000 บาท อีกทั้งหากการกระทำของคุณเป็นเหตุให้ผู้อื่นถึงแก่ความตาย คุณอาจถูกลงโทษจำคุกตั้งแต่ 5 ปีถึง 20 ปี และต้องรับโทษปรับตั้งแต่ 10,000 บาทถึง 40,000 บาท ดังนั้น ควรปฏิบัติตามกฎหมายอย่างถูกต้องและปฏิบัติตามหลักการและค่านิยมที่ดี เพื่อป้องกันตัวเองไม่ให้ผิดกฎหมาย


In [133]:
import os
class Chatbot():
    def __init__(self):
        self.memory = [{"role":"system","content":""},{"role":"user","content":""}]
    # def check_memory_file(self):  
    #     if os.path.isfile("./memory.txt"):  
    #         pass  
    #     else:  
    #         with open("./memory.txt", "w", encoding='utf-8') as mem:  
    #             mem.write(x)
    #             mem.close()
    # def clear_memory(self):
    #     # file_path = "/path/to/your/file.txt"  
    #     if os.path.isfile("./memory.txt"):  
    #         os.remove("./memory.txt")  
    #     else:
    #         print("Chatbot does not have memory.")
    def update_memory(self, role, content):  
        self.memory.append({"role": role, "content": content}) 
        
    def gpt3_sum(user_input, engine='gpt-35-turbo', temp=0.5, top_p=0.95, tokens=2000, freq_pen=0.0, pres_pen=0.2):
        setting = [{"role":"system","content":""},{"role":"user","content":""}]
        setting.append({'role':'system',"content":"Summary the user input as a bullet , use only thai language"})
        setting.append({'role':'user',"content":f"{user_input}"})
        response = openai.ChatCompletion.create(engine=engine, temperature=temp, max_tokens=tokens,
        top_p=top_p, frequency_penalty=freq_pen, presence_penalty=pres_pen, messages = setting)
        return response['choices'][0]['message']['content']  

    def gpt(user_input, engine='gpt-35-turbo-16k', temp=0.5, top_p=0.95, tokens=2000, freq_pen=0.0, pres_pen=0.2):
        update_setting('user', user_input)  
        lawyer_prompt_content = Lawyer_prompt + ''.join(search_vectordb(thai2eng(gpt3_sum(user_input))))  
        update_setting('system', lawyer_prompt_content)  
    
        response = openai.ChatCompletion.create(engine=engine, temperature=temp, max_tokens=tokens,  
        top_p=top_p, frequency_penalty=freq_pen, presence_penalty=pres_pen, messages = setting)  
        print(response['choices'][0]['message']['content'])

    def clear_memory(self):
        self.setting = [{"role":"system","content":""},{"role":"user","content":""}]


In [134]:
bot = Chatbot()  
bot.check_memory_file() 

In [135]:
bot.clear_memory()

In [41]:
conn.commit()
cursor.close()
conn.close()